In [ ]:
!pip install -q -U google-generativeai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.1/142.1 kB 4.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.6/663.6 kB 9.0 MB/s eta 0:00:00


In [ ]:
import pathlib
import textwrap
import time

import google.generativeai as genai

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

In [ ]:
# Used to securely store your API key
from google.colab import userdata

In [ ]:
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

In [ ]:
for m in genai.list_models():
  if 'generateContent' in m.supported_generation_methods:
    print(m.name)

models/gemini-1.0-pro
models/gemini-1.0-pro-001
models/gemini-1.0-pro-latest
models/gemini-1.0-pro-vision-latest
models/gemini-1.5-pro-latest
models/gemini-pro
models/gemini-pro-vision


In [ ]:
model = genai.GenerativeModel('gemini-pro')

In [ ]:
import pandas as pd
data = pd.read_csv('/content/Data_for_questioning.csv')

In [ ]:
# Convert DataFrame to JSON
data.to_json('final_data_json.json', orient='records', indent=4)

In [ ]:
import json
# Open the JSON file and load its contents
with open('/content/final_data_json.json', 'r') as f:
    data_json = json.load(f)

In [ ]:
data_json[0]

{'objectName': 'Chess set',
 'title': 'Chess set',
 'artistDisplayName': None,
 'objectDate': 'ca. 1800',
 'medium': 'Lacquered wood, ivory',
 'dimensions': 'Board: 1 3/4 x 16 5/8 x 16 5/8 in. (4.4 x 42.2 x 42.2 cm)',
 'city': None,
 'classification': None,
 'description': 'The game of chess probably originated in India, where ivory chess pieces have been found dating as far back as the eighth century. This set and board were made for the Western market but are distinguished from the majority of export pieces by their exquisitely intricate carving. The use of red and white figures instead of black and white is characteristic of sets produced in India.'}

In [ ]:
art_texts = []
i = 0

for data in data_json:
  prompt = f"Following is the description and meta data of an art. Based on this generate 4 questions and corresponding answer. Denote each question as **Question and answer as **Answer. {data}"
  response = model.generate_content(prompt)
  art_texts.append(response.text)
  # time.sleep(2)
  i += 1
  if i==60:
    print("60 completed")
    i=0
    time.sleep(10)


60 completed
60 completed
60 completed
60 completed
60 completed
60 completed
60 completed
60 completed
60 completed


In [ ]:
len(art_texts)

575

In [ ]:
to_markdown(art_texts[2])

> **Question 1:** What is the material and technique used in the creation of this art?
> **Answer:** Green glass, cut and inlaid with gold
> 
> **Question 2:** When was the art piece created?
> **Answer:** Early 19th century
> 
> **Question 3:** What are the dimensions of the art?
> **Answer:** H. 7 1/2 in. (19.1 cm), Diam. 6 1/4 in. (15.9 cm)
> 
> **Question 4:** What is the significance of this art in the museum's collection?
> **Answer:** It bridges two important parts of the collection: Indian decorative arts and objects evidencing cultural exchange with Europe in the eighteenth and nineteenth centuries.

In [ ]:
# Extracts questions and answers
def extract_qa_pairs(text):

  qa_pairs = {}
  current_q = 1
  current_a = 1

  for line in text.splitlines():
    if line.startswith("**Question"):
      qa_pairs[f"Question{current_q}"] = line.strip().split(":", 1)[1].strip("**")  # Extract question
      current_q += 1
    elif line.startswith(("**Answer", "Answer")):
      qa_pairs[f"Answer{current_a}"] = line.strip().split(":", 1)[1].strip("**")  # Extract answer
      current_a += 1

  return qa_pairs

In [ ]:
# Create an empty DataFrame with 4 questions and answers
df = pd.DataFrame(columns=["Question1", "Answer1", "Question2", "Answer2", "Question3", "Answer3", "Question4", "Answer4"])

In [ ]:
# Loop through each art text
t = 0
for art_text in art_texts:
  qa_dict = extract_qa_pairs(art_text)

  if len(list(qa_dict.values())) == 8:
    # Fill the DataFrame with extracted data
    df.loc[len(df)] = list(qa_dict.values())  # Efficiently fill with all values
  else:
    df.loc[len(df)] = [None]*8


In [ ]:
df

,Question1,Answer1,Question2,Answer2,Question3,Answer3,Question4,Answer4
0,What is the material used to create the chess...,Lacquered wood and ivory,When was the chess set likely produced?,Circa 1800,Where did the game of chess most likely origi...,India,What distinguishes this chess set from other ...,Its exquisitely intricate carving
1,What is the name of the art?,Watch stand,When was the art created?,1800–1820,What is the material used to create the stand?,Ivory,What is the purpose of the stand?,To display a pocket watch upright
2,What is the name of the object?,Huqqa Base,What materials were used to create the object?,"Green glass, cut and inlaid with gold",What is the approximate age of the object?,Early 19th century,How does the object bridge two important part...,It combines decorative floral forms from Indi...
3,What is the object's name and type?,"Armlet, One of a Pair",When was the armlet made?,19th century,What materials were used to create the armlet?,Silver; cast and chased,Where was the armlet produced?,"Chennai (Madras), southern India"
4,What is the name of the art object?,Water Container (Ganga-Jumna Lota),When was the object created?,19th century,What materials were used to make the object?,Brass and copper,Where was the object most likely made?,Thanjavur
...,...,...,...,...,...,...,...,...
570,What is the name of the art piece?,Shawl,To what century does the Shawl belong?,19th century,What material is used to make the Shawl?,Wool,What influenced the popularity of shawls in E...,The sophisticated male dress of the Mughal co...
571,What is the title of the illustration?,"""An Old Man and His Young Wife Before Religio...",In what year was the painting created?,1610–15,What technique was used to create the painting?,Opaque watercolors and gold on paper,Who is the artist of the painting?,Daulat
572,Who created the folio and during whose reign?,The folio was created during the reign of Emp...,What material and techniques were used to mak...,"Ink, opaque watercolor, and gold on paper wer...",What episode from Islamic history does the fo...,The folio depicts the execution of the rebel ...,What is the significance of the scene being d...,"The scene illustrates an episode from the ""Mi..."
573,,,,,,,,


In [ ]:
df.to_csv('QnA.csv', index=False)

In [ ]:
!pip install googletrans==3.1.0a0

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.1/55.1 kB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 133.4/133.4 kB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 4.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.6/53.6 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.0/65.0 kB 8.8 MB/s eta 0:00:00
  Created wheel for googletrans: filename=googletrans-3.1.0a0-py3-none-any.whl size=16353 sha256=fbcc1dfdc33ac517cd3adba39574902f88c38f9cf9b58cdbffcdc887bba4d60a
  Stored in directory: /root/.cache/pip/wheels/50/5d/3c/8477d0af4ca2b8b1308812c09f1930863caeebc762fe265a95
Successfully built googletrans
  Attempting uninstall: chardet
    Found existing installation: chardet 5.2.0
    Uninstalling c

In [ ]:
from googletrans import Translator

def translate_hi2en(sentence):
  """
  Function to translate from Hindi to English.

  Args:
   - sentence: string in Hindi

  Returns:
   - English translated text string

  """
  translator = Translator()
  output = translator.translate(sentence, dest='en', src='hi')
  return output.text

def translate_en2hi(sentence):
  """
  Function to translate from English to Hindi.

  Args:
   - sentence: string in English

  Returns:
   - Hindi translated text string

  """
  translator = Translator()
  output = translator.translate(sentence, dest='hi', src='en')
  return output.text

In [ ]:
sentence = "How does the object bridge two important parts of a collection?"

translate_en2hi(sentence)

'वस्तु किसी संग्रह के दो महत्वपूर्ण भागों को कैसे जोड़ती है?'

In [ ]:
sentence = "वस्तु किसी संग्रह के दो महत्वपूर्ण भागों को कैसे जोड़ती है?"
translate_hi2en(sentence)

'How does an object connect two important parts of a collection?'